In [1]:
from plotnine import *
import pandas
from solarized import *
import tikzplotlib
pandas.options.display.max_rows=10

In [2]:
# 1.{5end_to_end} tpch 1,2,3,4,5 with scale 1, 10
# 2 {5module4} and 3. module4 | also {5module4mem} and {5module4net}
# 3.{5tpchmodin} tpch 1, 4, 5 with modin on scale 10
# 4.{5micro_scales} micros with scale 1, 10 | also {5micro_traffic}
# 5.{5micro_net} micro join, selection with net=wan and scale 1

In [4]:
factors= ['wflow', 'optimizer', 'net', 'scale']
df=pandas.read_feather('specific-reports/5micro_scales.feather')
df.columns

Index(['bytes_memory', 'cpu_percent', 'bytes_sent', 'bytes_recv',
       'snapshot_idx', 'proc_time', 'exitcode', 'db_time', 'shared_wall_time',
       'wflow', 'optimizer', 'scale', 'net'],
      dtype='object')

# CHECK

In [5]:
df['exitcode'].value_counts()

 0     35673
-15     4461
Name: exitcode, dtype: int64

In [9]:
df.query("exitcode != 0")['wflow'].unique()

array(['micro_join.py'], dtype=object)

In [11]:
df.query("exitcode != 0")['optimizer'].unique()

array(['optimized'], dtype=object)

# PREPROCESS

In [12]:
df['gb_net'] = df['bytes_recv']/10**9

In [13]:
overview = df.groupby(by=factors).first().reset_index()
overview.columns

Index(['wflow', 'optimizer', 'net', 'scale', 'bytes_memory', 'cpu_percent',
       'bytes_sent', 'bytes_recv', 'snapshot_idx', 'proc_time', 'exitcode',
       'db_time', 'shared_wall_time', 'gb_net'],
      dtype='object')

In [ ]:
plot_df = overview

# {5micro_scales}

In [ ]:
plot = (
        ggplot(plot_df, aes('wflow', y='shared_wall_time', fill='optimizer'))
        + geom_col(width = 0.5, position = "dodge")
        #+ scale_fill_manual(values=cdict) 
        + scale_y_sqrt()
        + labs(fill="Optimization Method")
        + facet_wrap('~ scale', scales='free_y', nrow=1, dir='v')
        + xlab("Query")
        + ylab("Time (in sec.)")
       # + theme(figure_size=(8, 2))
               #subplots_adjust={'hspace': 1, 'wspace': 0.25})
        + theme(figure_size=(8, 2), subplots_adjust={'wspace': 0.20})
    )

In [ ]:
plot.save("specific-plots/5end_to_end.png")
matplot = plot.draw()
tikzplotlib.save('specific-plots/5end_to_end.tikz', figure=matplot)
display(plot)